In [20]:
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import os
import json
import re
import datetime

from seleniumwire import webdriver  # Import from seleniumwire


import pandas as pd

In [13]:
driver = webdriver.Chrome() 


In [21]:
# For mouse scroll
def wheel_element(element, deltaX = 0, deltaY = 120, offsetX = 0, offsetY = 0):
    error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaX = arguments[1]
    var deltaY = arguments[2];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[3] || box.width / 2);
    var clientY = box.top + (arguments[4] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaX: deltaX, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaX, deltaY, offsetY)

def get_websocket_data(ticker, interval):
    # Initiate a chrome web driver
    chrm_options = Options()
    chrm_caps = webdriver.DesiredCapabilities.CHROME.copy()
    chrm_caps['goog:loggingPrefs'] = { 'performance':'ALL' } #save all returns including websocket data
    driver = webdriver.Chrome(chrome_options=chrm_options,
                              desired_capabilities=chrm_caps) 

    driver.maximize_window()
    # Go to Trading View
    website = 'https://www.tradingview.com/'
    driver.get(website)
    
    # Input query
    query = driver.find_element_by_name('query')
    query.send_keys(ticker)
    query.submit()
    driver.implicitly_wait(1)
    
    # Select frequency
    control = driver.find_element_by_id('header-toolbar-intervals')
    control.click()
    
    if interval == "1h":
        select_1h = driver.find_element_by_xpath("//div[@data-value='60']")
        select_1h.click()
        
    # Scroll the chart to retrieve all data    
    elm = driver.find_element_by_class_name('chart-gui-wrapper')
    for i in range(100):
        wheel_element(elm, deltaX=0, deltaY=100)

    for i in range(1000):
        wheel_element(elm, deltaX=-10, deltaY=0)
    
#     driver.close()
    return driver.requests

def parse_driver_requests(driver_requests):
    ws = []
    for request in driver_requests:
        if request.ws_messages:
            ws.append(request.ws_messages)
            
    messages = []
    for w in ws:
        for l in w:
            m = l.content
            txt_list = re.split('~m~\d*~m~',m)       
            for t in txt_list:
                if len(t)<10:
                    continue
                t = json.loads(t)
                try:
                    if t['m'] == 'timescale_update':
                        messages.append(t)
                except:
                    continue
                    
    price_data = []

    for m in messages:
        try:
            _j = m['p'][1]
            price_data.append([s['v']  for s in _j['sds_1']['s']])
        except:
            continue
            
    if len(price_data)==0:
        print('ticker not avaliable')
        return None
    df = [pd.DataFrame(p) for p in price_data]
    df = pd.concat(df)

    df.columns = ['Date', 'open', 'high', 'low', 'close', 'volume']
    df.Date = pd.to_datetime(df.Date, unit='s')
    
    
    print('before drop duplicate', len(df))
    df = df.drop_duplicates('Date')
    df.sort_values('Date', inplace=True)
    print('after',len(df))
    df.reset_index(inplace=True, drop=True)
    return df

In [22]:
driver_requests = get_websocket_data('BSVUSD', '1h')
data = parse_driver_requests(driver_requests)

before drop duplicate 26692
after 13231


In [23]:
data.tail(50)

,Date,open,high,low,close,volume
13181,2021-07-06 23:00:00,139.02,140.96,138.86,140.88,45.796387
13182,2021-07-07 00:00:00,140.95,147.19,139.94,139.94,1557.860785
13183,2021-07-07 01:00:00,140.96,141.56,140.53,140.94,19.335883
13184,2021-07-07 02:00:00,141.21,141.92,140.60,141.92,43.346523
13185,2021-07-07 03:00:00,142.02,143.20,142.02,143.20,58.316652
13186,2021-07-07 04:00:00,143.24,143.91,142.34,143.73,78.099763
13187,2021-07-07 05:00:00,143.61,143.83,143.20,143.59,83.453691
13188,2021-07-07 06:00:00,143.64,144.09,143.14,143.47,64.256894
13189,2021-07-07 07:00:00,143.45,144.80,143.42,143.42,53.815715
13190,2021-07-07 08:00:00,143.65,144.86,143.65,144.25,80.549365


In [24]:
data.to_csv('Bitfinex_bsv.csv')

In [18]:
data.head(50)

,Date,open,high,low,close,volume
0,2020-01-01 00:00:00,13.661320,13.677448,13.632525,13.643193,42633.057629
1,2020-01-01 01:00:00,13.649882,13.768640,13.641589,13.752325,48151.218573
2,2020-01-01 02:00:00,13.759720,13.811889,13.740092,13.805787,36351.274554
3,2020-01-01 03:00:00,13.805955,13.807399,13.741912,13.761379,37446.988073
4,2020-01-01 04:00:00,13.762916,13.774633,13.723174,13.741227,50573.570818
5,2020-01-01 05:00:00,13.746958,13.766478,13.734050,13.747412,14451.927306
6,2020-01-01 06:00:00,13.750048,13.796293,13.742552,13.775900,21654.487657
7,2020-01-01 07:00:00,13.780145,13.790746,13.693256,13.716571,27128.910256
8,2020-01-01 08:00:00,13.710943,13.717738,13.653556,13.683563,29074.885609
9,2020-01-01 09:00:00,13.690435,13.700355,13.638544,13.670523,33122.965477
